# Ectopy: Ectopic expression analysis in Python

The code below presents a typical pipeline of `ectopy` package to calculate activation frequencies of ectopically expressed genes and to find potential prognostic biomarkers.

## Import data

In [5]:
import pandas as pd
data_dir = 'data/'
data = pd.read_csv(data_dir + 'data.csv', sep=';', index_col='id_sample')
expgroup = pd.read_csv(data_dir + 'expgroup.csv', sep=';', index_col='id_sample')

Display data.

In [6]:
print('Data', data.shape)
data.head(3)

Data (1144, 3)


,DNMT3B,EXO1,MCM10
id_sample,,,
TCGA-3C-AAAU-01A,2.293107,1.851411,1.472214
TCGA-3C-AALI-01A,2.213523,2.893140,2.099413
TCGA-3C-AALJ-01A,3.160032,1.735396,1.655712


Display a list of available genes.

In [7]:
genes = list(data.columns)
print('Available genes', genes)

Available genes ['DNMT3B', 'EXO1', 'MCM10']


Display expgroup with tissue status (column `group`) and survival data (columns `time` and `event`).

In [8]:
print('Expgroup', expgroup.shape)
expgroup.head(3)

Expgroup (1144, 3)


,group,time,event
id_sample,,,
TCGA-3C-AAAU-01A,tumoral,134.90,0.0
TCGA-3C-AALI-01A,tumoral,133.50,0.0
TCGA-3C-AALJ-01A,tumoral,49.13,0.0


## Create normal and tumoral datasets

In [9]:
expgroup_normal = expgroup[expgroup['group']=='normal']
expgroup_tumoral = expgroup[expgroup['group']=='tumoral']
normal = data.loc[expgroup_normal.index, :]
tumoral = data.loc[expgroup_tumoral.index, :]

In [10]:
print('Tumoral:', 'data', tumoral.shape, 'expgroup', expgroup_tumoral.shape)
print('Normal:', 'data', normal.shape, 'expgroup', expgroup_normal.shape)

Tumoral: data (1047, 3) expgroup (1047, 3)
Normal: data (97, 3) expgroup (97, 3)


## Calculate m2sd threshold

In [11]:
from analysis import threshold
m2sd_threshold = threshold.StdDecorator(threshold.MeanTreshold(normal), nb_std=2).calculate_threshold()
print('Threshold m2sd')
print(m2sd_threshold)

Threshold m2sd
DNMT3B    1.396983
EXO1      1.089280
MCM10     0.898020
dtype: float64


## Calculate frequencies of expression above a threshold

In [12]:
from analysis import expression_analysis
frequency = expression_analysis.ExpressionFrequency().calculate_expression_frequency(tumoral, m2sd_threshold)
print('Activation frequency obtained with m2sd threshold in percentage')
print(frequency.head())

Activation frequency obtained with m2sd threshold in percentage
DNMT3B    45.367717
EXO1      81.088825
MCM10     73.925501
dtype: float64


## Calculate adaptive threshold

In [13]:
from analysis import threshold

In [16]:
max_normal = threshold.MaxTreshold(normal).calculate_threshold()

In [17]:
options = {
    'percentile': 15.0,
    'step_percentile': 1.0,
    'min_nb_samples': 20,
    'noise_level': 0.3,
    'min_reference_threshold': max_normal,
    'nb_folds': 4,
    'nb_cross_validations': 1
    }


In [20]:
adaptive_threshold = threshold.AdaptiveThreshold(data=tumoral, survival_data=expgroup_tumoral, duration_col='time', event_col='event', **options)
adaptive = adaptive_threshold.calculate_threshold()
print(list(adaptive))

Processing feature DNMT3B
Optimal threshold for DNMT3B
     threshold  threshold_percentile   p_value  hazard_ratio  validated  \
T11   1.787853             72.464183  0.003263      1.557107       True   

     cv_score  optimal  
T11      25.0     True  
Processing feature EXO1
Optimal threshold for EXO1
     threshold  threshold_percentile   p_value  hazard_ratio  validated  \
T15   2.188157             60.704871  0.009725      1.459836       True   

     cv_score  optimal  
T15      25.0     True  
Processing feature MCM10
Optimal threshold for MCM10
    threshold  threshold_percentile   p_value  hazard_ratio  validated  \
T5   1.655761             57.486151  0.003297      1.537064       True   

    cv_score  optimal  
T5      25.0     True  
[1.7878532762177646, 2.188157291575931, 1.6557611272970392]


In [21]:
print(adaptive_threshold.get_details('EXO1').head(15))

     threshold  threshold_percentile   p_value  hazard_ratio  validated  \
T1    1.813697             46.704871  0.025297      1.396049       True   
T2    1.846114             47.704871  0.014063      1.442413       True   
T3    1.860113             48.704871  0.006309      1.501328       True   
T4    1.899769             49.704871  0.011484      1.452290       True   
T5    1.921833             50.704871  0.008367      1.474996       True   
T6    1.945113             51.704871  0.007588      1.481416       True   
T7    1.964816             52.704871  0.002709      1.554961       True   
T8    2.003815             53.704871  0.003130      1.542375       True   
T9    2.024925             54.704871  0.003049      1.543227       True   
T10   2.053258             55.704871  0.002321      1.561623       True   
T11   2.075781             56.704871  0.001055      1.614978       True   
T12   2.110568             57.704871  0.002459      1.556553       True   
T13   2.126367           